## Collision élastique de deux disques

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import Image, HTML

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, curdoc
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider

output_notebook()

In [ ]:
Image("elastic_coll.png")

In [ ]:
# fonction calcul de la masse
mass = lambda P : (4 / 3) * P['density'] * np.pi * P['radius'] ** 3

vel_ = lambda P: (P['v']*np.cos(P['alpha']*np.pi/180), P['v']*np.sin(P['alpha']*np.pi/180))

traj_ = lambda P:  [P['x'] + P['v_x']*np.linspace(0, 500, 500)/10, P['y'] + P['v_y']*np.linspace(0, 500, 500)/10]

new_ = lambda P, dt: [P['x'] + P['v_x']*dt, P['y'] + P['v_y']*dt]

dist_ = lambda P1, P2: math.sqrt((P1['x'] + P2['x'])**2 + (P1['y'] + P2['y'])**2)

proj_v = lambda P, w: np.array([P['v_x']*w[0] + P['v_y']*w[1]])/P['v'] * w

new_v_coll = lambda P, w, u: [proj_v(P,w) + (np.array([P['v_x'], P['v_y']]) - proj_v(P,u))]

In [ ]:


def update_values_time(P1, P2, dt):
    # calculer prochain possition
    P1['x'], P1['y'] = new_(P1)
    P2['x'], P2['y'] = new_(P2)
    
    # verifier si il y haura une collision
    if dist_(P1, P2) < (P1['radius'] + P2['radius']):
        
        # calculer vecteur perpendiculaire à la collision
        u = np.array([P2['x'] - P1['x'], P2['y'] - P1['y']])/dist_(P1,P2)
        
        # point collision
        p_x = P1['x'] + u[0]*P1['radius']/dist_(P1,P2)
        p_y = P1['y'] + u[1]*P1['radius']/dist_(P1,P2)
        
        # vecteur tangent à la collision
        w = np.array([u[1], -u[0]])
        
        # composant velocité perpendiculaire à la collision
        #v_w_P1 = proj_v(P1, w)
        #v_w_P2 = proj_v(P2, w)
        
        # composant velocité tangent à la collision
        #v_u_P1 = proj_v(P1, u)
        #v_u_P2 = proj_v(P2, u)
        
        P1['v_x'], P1['v_y'] = new_v_coll(P1, w, u)
        P2['v_x'], P2['v_y'] = new_v_coll(P2, w, u)
        
        #P1['v'] = math.sqrt(P1['v_x']**2 + P1['v_y']**2)
        #P2['v'] = math.sqrt(P2['v_x']**2 + P2['v_y']**2)
        
    P1['t'] -= dt
    P2['t'] -= dt
    

def update_params(P, P_y, P_radius, v_P, alpha_P):
    P['y'] = P_y
    P['v'] = v_P
    
    if P['name'] == 'A':
        P['alpha'] = alpha_P -90
    else:
        P['alpha'] = alpha_P + 90
    
    P['radius'] = P_radius
    P['m'] = mass(P)
    P['v_x'], P['v_y'] = vel_(P)
    P['traj'] = traj_(P)
    
    

    

# Initialize
A = {'x':-250, 'y': 0, 'v':25, 'alpha': 10, 'density': 1, 'radius': 5, 't':10, 'name':'A'}
B = {'x':250, 'y': 0, 'v':15, 'alpha': 30, 'density': 1, 'radius': 10, 't':10, 'name':'B'}


A['v_x'], A['v_y'] = vel_(A) 
B['v_x'], B['v_y'] = vel_(B) 

A['m'], B['m'] = mass(A), mass(B)

A['traj'] = traj_(A)
B['traj'] = traj_(B)

In [ ]:
# Figure
p = figure(title="Collision", plot_height=450, plot_width=900, y_range=(-100,100), x_range=(-300,300), 
           background_fill_color='#ffffff')

# grille
p.ygrid.minor_grid_line_color = 'grey'
p.ygrid.minor_grid_line_alpha = 0.3

p.xgrid.minor_grid_line_color = 'grey'
p.xgrid.minor_grid_line_alpha = 0.3

# point initial
p_A = p.circle([A['x']], [A['y']], size=A['radius'], fill_color='#e32020', line_color='#e32020', legend='A')
p_B = p.circle([B['x']], [B['y']], size=B['radius'], fill_color='#0ABDE3', line_color='#0ABDE3', legend='B')

# centre de masse
p_centre_A =  p.circle([A['x']], [A['y']], size=2, fill_color='#000000', line_color='#000000')
p_centre_B = p.circle([B['x']], [B['y']], size=2, fill_color='#000000', line_color='#000000')

g_pA = p_A.glyph
g_pB = p_B.glyph

# trajetoires
t_A = p.line(A['traj'][0], A['traj'][1], color='#747b7c')
t_B = p.line(B['traj'][0], B['traj'][1], color='#747b7c')


def update(A_y, B_y, A_radius, B_radius, v_A, v_B, alpha_A, alpha_B):
    
    update_params(A, A_y, A_radius, v_A, alpha_A)
    update_params(B, B_y, B_radius, v_B, alpha_B)
    

    # mettre à jour radius
    g_pA.size = A['radius']
    g_pB.size = B['radius']
    
    p_centre_A.data_source.data['y'] = [A['y']]
    p_centre_B.data_source.data['y'] = [B['y']]
    
    # mettre à jour position
    g_pA.y = A['y']
    g_pB.y = B['y']
    
    # mettre à jour trajectoire
    t_A.data_source.data['x'] = A['traj'][0]
    t_A.data_source.data['y'] = A['traj'][1]
    t_B.data_source.data['x'] = B['traj'][0]
    t_B.data_source.data['y'] = B['traj'][1]
    
    print('Masse point A: {:0.2f} kg'.format(A['m']/1000))
    print('Masse point B: {:0.2f} kg'.format(B['m']/1000))
    
    push_notebook()
    
slider_Ay = widgets.IntSlider(min=-90, max=90, step=5, value=0, description='$A_{y} [m]$:')
slider_By = widgets.IntSlider(min=-90, max=90, step=5, value=0, description='$B_{y} [m]$:')

slider_Ar = widgets.FloatSlider(min=5,max=50,step=1,value=10, description='$A_{radius} [m]$:')
slider_Br = widgets.FloatSlider(min=5,max=50,step=1,value=10, description='$B_{radius} [m]$:')

slider_Av = widgets.FloatSlider(min=2,max=50,step=1,value=10, description='$v_{A} [m/s]$:')
slider_Bv = widgets.FloatSlider(min=2,max=50,step=1,value=15, description='$v_{B} [m/s]$:')

slider_A_alpha = widgets.FloatSlider(min=0,max=180,step=1,value=0, description='$ alpha A [º]$:')
slider_B_alpha = widgets.FloatSlider(min=0,max=180,step=1,value=0, description='$ alpha B [º]$:')

"""

slider_Ay = Slider(title="$A_{y}$ [m]", start=-90, end=90, step=1, value=0)
slider_By = Slider(title="$B_{y}$ [m]", start=-90, end=90, step=1, value=0)

slider_Ar = Slider(title="$R_{A}$ [m]", start=2, end=50, step=1, value=10)
slider_Br = Slider(title="$R_{B}$ [m]", start=2, end=50, step=1, value=10)

slider_Av = Slider(title="$V_{B}$ [m/s]", start=0, end=90, step=1, value=5)
slider_Bv = Slider(title="$V_{B}$ [m/s]", start=0, end=90, step=1, value=10)

slider_A_alp = Slider(title="$Alpha_{A}$ [º]", start=0, end=180, step=1, value=30)
slider_B_alp = Slider(title="$Alpha_{A}$ [º]", start=0, end=180, step=1, value=40)

"""

show(p, notebook_handle=True)
interact(update, A_y = slider_Ay, B_y = slider_By, A_radius = slider_Ar, B_radius = slider_Br,\
        v_A = slider_Av, v_B = slider_Bv, alpha_A = slider_A_alpha, alpha_B = slider_B_alpha);



In [ ]:
# TODO: 

# GENERATE VIDEO FROM INITIAL CONF GOT FROM THE INTERACTIVE PLOT
# GENERATE WITH FUNCTION UPDATE_VALUES_TIME BY ITERATING UNTIL T<= 0
# SHOULD IT CHECK COLLISIONS AGAINST WALLS?
